In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
from glove import Corpus, Glove

In [3]:
model = Glove.load('../../Dataset/glove.bin')

In [4]:
data = pd.read_csv('../../Dataset/Local_database_figer.csv')

In [5]:
len(data)

3019617

In [6]:
MFT_data=pd.read_csv('../../Dataset/Local_MFT_dataset.csv')
LFT_data=pd.read_csv('../../Dataset/Local_LFT_dataset.csv')

In [5]:
data.head()

,Unnamed: 0,sentence_ID,entity_name,fine_grain,sentence,coarse_grain
0,0,15,European Union,"['/people/ethnicity', '/location/country', '/l...",112 Emergency phone number across the European...,/people
1,1,15,European Union,"['/people/ethnicity', '/location/country', '/l...",112 Emergency phone number across the European...,/location
2,2,15,European Union,"['/people/ethnicity', '/location/country', '/l...",112 Emergency phone number across the European...,/person
3,3,44,Stockton and Darlington Railway,"['/rail/railway', '/organization', '/organizat...",The first 0-4-0 to use coupling rods was Locom...,/rail
4,4,44,Stockton and Darlington Railway,"['/rail/railway', '/organization', '/organizat...",The first 0-4-0 to use coupling rods was Locom...,/organization


In [7]:
def preprocessing(dataset):
    
    X, y = np.asarray(dataset['sentence']), np.asarray(dataset['coarse_grain'])
    label_map = {cat:index for index,cat in enumerate(np.unique(y))}
    y_prep = np.asarray([label_map[l] for l in y])
    x_tokenized = [[w for w in sentence.split(" ") if w != ""] for sentence in X]
   
    return x_tokenized, y_prep

In [8]:
class Sequencer():
    
    def __init__(self,
                 all_words,
                 max_words,
                 seq_len,
                 model
                ):
        
        self.seq_len = seq_len
        self.glove_model = model
        """
        temp_vocab = Vocab which has all the unique words
        self.vocab = Our last vocab which has only most used N words.
    
        """
        temp_vocab = list(set(all_words))
        self.vocab = []
        self.word_cnts = {}
        """
        Now we'll create a hash map (dict) which includes words and their occurencies
        """
        for word in temp_vocab:
            # 0 does not have a meaning, you can add the word to the list
            # or something different.
            count = len([0 for w in all_words if w == word])
            self.word_cnts[word] = count
            counts = list(self.word_cnts.values())
            indexes = list(range(len(counts)))
        
        # Now we'll sort counts and while sorting them also will sort indexes.
        # We'll use those indexes to find most used N word.
        cnt = 0
        while cnt + 1 != len(counts):
            cnt = 0
            for i in range(len(counts)-1):
                if counts[i] < counts[i+1]:
                    counts[i+1],counts[i] = counts[i],counts[i+1]
                    indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
                else:
                    cnt += 1
        
        for ind in indexes[:max_words]:
            self.vocab.append(temp_vocab[ind])
                    
    def textToVector(self,text):
        # First we need to split the text into its tokens and learn the length
        # If length is shorter than the max len we'll add some spaces (100D vectors which has only zero values)
        # If it's longer than the max len we'll trim from the end.
        tokens = text.split()
        len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
        vec = []
        for tok in tokens[:len_v]:
            try:
                dict_index = self.glove_model.dictionary[tok]
                vec.append(self.glove_model.word_vectors[dict_index])
            except Exception as E:
                pass
        
        last_pieces = self.seq_len - len(vec)
        for i in range(last_pieces):
            vec.append(np.zeros(5,))
        
        return np.asarray(vec).flatten()

# 1. For whole dataset

In [40]:
data = data.sample(n=80000)

In [41]:
x_tokenized, y_prep = preprocessing(data)

In [42]:
Entities = np.asarray(MFT_data['entity_name'])
e_tokenized = [[w for w in str(e).split(" ") if w != ""] for e in Entities]

In [43]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 1200,
              seq_len = 15,
              model = model
             )

In [38]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

Sum of variance ratios:  0.7340833652157553


In [39]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=50)
pca_model.fit(e_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))

In [40]:
e_comps = pca_model.transform(e_vecs)

In [41]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 10,
              seq_len = 5,
              model = model
             )

In [42]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

In [44]:
x_train,x_test,y_train,y_test = train_test_split(e_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(64000, 50)
(16000, 50)
(64000,)
(16000,)


In [45]:
svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [46]:
y_pred = svm_classifier.predict(x_test)

In [47]:
from sklearn.metrics import f1_score

In [48]:
f1_score(y_test, y_pred, average='macro')

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.010370053475935831

In [49]:
f1_score(y_test, y_pred, average='micro')

0.329375

In [50]:
f1_score(y_test, y_pred, average='weighted')

0.16368164064171126

# 2. For MFT

In [25]:
MFT_data = MFT_data.sample(n=80000)

In [26]:
x_tokenized, y_prep = preprocessing(MFT_data)

In [28]:
Entities = np.asarray(MFT_data['entity_name'])
e_tokenized = [[w for w in str(e).split(" ") if w != ""] for e in Entities]

In [29]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 1200,
              seq_len = 15,
              model=model
             )

In [30]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

In [31]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=50)
pca_model.fit(e_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))

Sum of variance ratios:  0.99891551711262


In [32]:
e_comps = pca_model.transform(e_vecs)

In [33]:
x_train,x_test,y_train,y_test = train_test_split(e_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(64000, 50)
(16000, 50)
(64000,)
(16000,)


In [34]:
svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [35]:
y_pred = svm_classifier.predict(x_test)

In [36]:
from sklearn.metrics import f1_score

In [37]:
f1_score(y_test, y_pred, average='macro')

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.054022166915521676

In [38]:
f1_score(y_test, y_pred, average='micro')

0.3605

In [39]:
f1_score(y_test, y_pred, average='weighted')

0.2300182029069613

# 3. For LFT

In [10]:
x_tokenized, y_prep = preprocessing(LFT_data)

In [12]:
Entities = np.asarray(LFT_data['entity_name'])
e_tokenized = [[w for w in str(e).split(" ") if w != ""] for e in Entities]

In [13]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 1200,
              seq_len = 15,
              model=model
             )

In [14]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

In [16]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=50)
pca_model.fit(e_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))

Sum of variance ratios:  0.9980737531391926


In [17]:
e_comps = pca_model.transform(e_vecs)

In [18]:
x_train,x_test,y_train,y_test = train_test_split(e_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(82147, 50)
(20537, 50)
(82147,)
(20537,)


In [19]:
svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [20]:
y_pred = svm_classifier.predict(x_test)

In [21]:
from sklearn.metrics import f1_score

In [22]:
f1_score(y_test, y_pred, average='macro')

/home/evelyn/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.06132819329696024

In [23]:
f1_score(y_test, y_pred, average='micro')

0.13711837171933583

In [24]:
f1_score(y_test, y_pred, average='weighted')

0.0947571713600634